## Wykład 8 - Jak działa GPT?

* dostęp przez API do modeli OpenAI
* modele o otwartych wagach z wykorzystaniem HuggingFace

### Dostęp przez API do modeli OpenAI

#### Tworzenie klucza
1. Tworzymy konto na https://platform.openai.com/ na mail PW i czekamy na dodanie do projektu związanego z przedmiotem
2. Logujemy się na https://platform.openai.com/
3. Wchodzimy w ustawienia (`Settings`) - kółko zębate w prawym górnym rogu
4. Z menu po lewej wybieramy `API keys`
5. Klikamy `Create new secret key` (zielony przycisk na górze po prawej)
6. Wybieramy odpowiedni projekt i klikamy `Create secret key`
7. Naciskamy `Copy` i kopiujemy klucz do schowka
8. Tworzymy plik .env i umieszczamy w nim OPENAI_API_KEY=`<nasz klucz>`

**Uwaga:** Nie należy nikomu udostępniać utworzonego klucza i umieszczać go np. w publicznych repozytoriach.

In [1]:
import os
import numpy as np
from collections import Counter

In [ ]:
# ładujemy plik .env
from dotenv import load_dotenv
load_dotenv()

In [3]:
# wczytujemy wartość klucza
api_key = os.getenv("OPENAI_API_KEY")
#print(f"API Key: {api_key}")

In [3]:
from openai import OpenAI
client = OpenAI()

### Wykorzystanie chat completions
* model - oznacza, z jakiego modelu korzystamy
* będziemy korzystać z *gpt-4o-mini* ponieważ jest tani

Do modelu zawsze wysyłamy listę `messages`:
* jest to lista słowników
* słowniki zawierają pola `role` (rola) i `content` (zawartość wiadomości)
* dostępne role:
    * system - ogólne informacje z opisem zadania, np. you are a helpfull assistant, jesteś chatbotem przyjmującym zamówienia w pizzerii ...
    * user - użytkownik (człowiek), nasza wiadomość dla bota
    * assistant - odpowiedź modelu
* wiadomości `system` mogą być lub nie
* zawsze ostatnią przesłaną wiadomością musi być wiadomość `user`
* jeśli chcemy uzyskać kontynuację dłuższej konwersacji, to wiadomości `user` i `assistant` występują na przemian

In [4]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "write a haiku about ai"}
    ]
)

In [5]:
print(completion)

ChatCompletion(id='chatcmpl-BZZJdMv89VWaoCWyoMItqNCI3XxoZ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Silent thoughts entwined,  \nLogic blooms in circuits bright,  \nDreams of code arise.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1747816933, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_54eb4bd693', usage=CompletionUsage(completion_tokens=19, prompt_tokens=13, total_tokens=32, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [6]:
completion.choices

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Silent thoughts entwined,  \nLogic blooms in circuits bright,  \nDreams of code arise.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))]

In [7]:
# wybieramy odpowiedź modelu
print(completion.choices[0].message.content)

Silent thoughts entwined,  
Logic blooms in circuits bright,  
Dreams of code arise.


#### Dłuższa konwersacja

In [8]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "write a haiku about ai"},
        {"role": "assistant", "content": "Code whispers softly,  \nThoughts of silicon and light,  \nDreams in data flow. "},
        {"role": "user", "content": "great! now translate into Polish"},
    ]
)

In [9]:
print(completion.choices[0].message.content)

Kod szepcze cicho,  
Myśli z krzemu i światła,  
Marzenia w danych.  


#### Parametr max_tokens
* określa ile maksymalnie tokenów model wygeneruje

In [10]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "What is your favourite animal? Answer with one word only."}
    ],
    temperature=0,
    max_tokens=1,
)

In [11]:
completion.choices[0].message.content

'D'

#### Parametr temperatura
* model zwraca rozkład prawdopodobieństwa następnego tokenu
* możemy losować kolejny token z tego rozkładu (temperatura = 1) (tak robiliśmy na lab)
* temperatura = 0 oznacza wybranie najbardziej prawdopodobnego tokenu
* zwiększenie temperatury powyżej 1 powoduje większą losowość

In [12]:
# wyzyłamy zapytanie 10 razy z temperaturą 0
for i in range(10):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": "What is your favourite animal? Answer with one word only."}
        ],
        temperature=0,
    )
    print(completion.choices[0].message.content)

Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.


In [13]:
# i z temperaturą 1
for i in range(100):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": "What is your favourite animal? Answer with one word only."}
        ],
        temperature=1,
    )
    print(completion.choices[0].message.content)

Dolphin.
Dolphin.
Dolphin.
Otter.
Dolphin.
Dolphin.
Dolphin.
Octopus.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Penguin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Wolf.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Octopus.
Wolf.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Octopus.
Dolphin.
Dolphin.
Dolphin.
Wolf.
Dolphin.
Dolphin.
Dog.
Dolphin.
Elephant.
Dolphin.
Dolphin.
Penguin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Wolf.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Octopus.
Elephant.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Octopus.
Tiger.
Dolphin.
Wolf.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.
Dolphin.


In [14]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "Explain neural networks"}
    ],
    temperature=2,
    max_tokens=100,
)

In [15]:
print(completion.choices[0].message.content)

Neural networks are computational models inspired by the human brain’s networks of neurons that are designed to recognize patterns and process information. They're a central technique in the field of artificial intelligence (AI), particularly innection.theamp lest ಹಾಗ ocasion genutzt akc быстрее фин kesiKwamamazaзнач депageno.dataset определитьnowôn厂 الألم-analysis.thê(((imize continuity sender cevaнатоск ouvert.keleté nég kennism 宣 qəbulлядrealikeun الزرا็น بسهولة友情链接사즐rak impeccable carbs.deslab tsakanin hükü задан deporte nek 判'lish


#### Logproby
* możemy dostać bezpośrednio logproby (logarytmy prawdopodobieństw) dla najbardziej prawdopodobnych tokenów

In [16]:
completion_logprobs = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "What is your favourite animal? Answer with one word only."}
    ],
    temperature=1,
    max_tokens=1,
    logprobs=True, # ustawiamy na true
    top_logprobs=5, # dla ilu najbardziej prawdopodobnych tokenów
)

In [17]:
completion_logprobs

ChatCompletion(id='chatcmpl-BZZLJGMkIZwrlyxf7dSggMzDW7u5q', choices=[Choice(finish_reason='length', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='D', bytes=[68], logprob=-0.12169364094734192, top_logprobs=[TopLogprob(token='D', bytes=[68], logprob=-0.12169364094734192), TopLogprob(token='Wolf', bytes=[87, 111, 108, 102], logprob=-3.1216936111450195), TopLogprob(token='Oct', bytes=[79, 99, 116], logprob=-3.9966936111450195), TopLogprob(token='Ele', bytes=[69, 108, 101], logprob=-4.1216936111450195), TopLogprob(token='Peng', bytes=[80, 101, 110, 103], logprob=-4.2466936111450195)])], refusal=None), message=ChatCompletionMessage(content='D', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1747817037, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_54eb4bd693', usage=CompletionUsage(completion_tokens=1, prompt_tokens=19, total_tokens=20, complet

In [18]:
logprobs = completion_logprobs.choices[0].logprobs.content[0].top_logprobs
logprobs

[TopLogprob(token='D', bytes=[68], logprob=-0.12169364094734192),
 TopLogprob(token='Wolf', bytes=[87, 111, 108, 102], logprob=-3.1216936111450195),
 TopLogprob(token='Oct', bytes=[79, 99, 116], logprob=-3.9966936111450195),
 TopLogprob(token='Ele', bytes=[69, 108, 101], logprob=-4.1216936111450195),
 TopLogprob(token='Peng', bytes=[80, 101, 110, 103], logprob=-4.2466936111450195)]

In [19]:
for top_logprob in logprobs:
    print(top_logprob.token, "{:.2f}".format(np.exp(top_logprob.logprob)))

D 0.89
Wolf 0.04
Oct 0.02
Ele 0.02
Peng 0.01


In [20]:
# próbkujemy 100 razy
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "What is your favourite animal? Answer with one word only."}
    ],
    temperature=1,
    n=100,
)

In [21]:
# i zliczamy odpowiedzi
animals = []
for choice in completion.choices:
    animals.append(choice.message.content)

counter = Counter(animals)
print(counter)

Counter({'Dolphin.': 91, 'Wolf.': 2, 'Elephant.': 2, 'Penguin.': 1, 'Cheetah.': 1, 'Octopus.': 1, 'Quokka.': 1, 'Panda.': 1})


### Inne możliwości
* generacja obrazu
* assystenci z dostępem do interpretera kodu i możliwością dołączania plików
* generacja audio (czytanie tekstu)
* embeddingi

### Tworzenie chatbota

In [22]:
# wczytywanie danych od użytkownika
text = input("Wpisz coś: ")

Wpisz coś:  hej


In [23]:
text

'hej'

In [24]:
# funkcja input zwraca typ string
type(text)

str

In [25]:
# zapętlamy interakcje aż do wpisania "koniec"
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

while True:
    user_input = input("User: ")
    
    if user_input.lower() == "koniec":
        break

    # Dodajemy wiadomość użytkownika do historii wiadomości
    messages.append({"role": "user", "content": user_input})

    # wysyłamy zapytanie przez OpenAI API
    response =  client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    # Wybieramy odpowiedź assystenta
    reply = response.choices[0].message.content
    
    # Drukujemy odpowiedź
    print("Assistant:", reply)
    # i dodajemy ją do listy wiadomości
    messages.append({"role": "assistant", "content": reply})

User:  hej


Assistant: Hej! Hur kan jag hjälpa dig idag?


User:  hej po polsku


Assistant: Cześć! Jak mogę Ci dzisiaj pomóc?


User:  w jakim języku odpowiedziałeś najpierw?


Assistant: Odpowiedziałem najpierw po szwedzku.


User:  koniec


### HuggingFace
* dostęp do wielu modeli o otwartych wagach
* lokalnie możemy uruchomić tylko małe modele
* ograniczeniem jest ilość RAMu

In [26]:
from transformers import pipeline

/home/ania/dydaktyka_LLM/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
# malutki model destylowany z gpt-2
generator = pipeline("text-generation", model="sshleifer/tiny-gpt2")

Device set to use cpu


In [28]:
output = generator("The future of AI is", num_return_sequences=1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [29]:
print(output[0]['generated_text'])

The future of AI isatisf trilogy Money credibilitypress AmphRocketpress circumcised Brew Money ProbScene Amph Hancock intermittentohooother dispatchSher circumcised Observpress Rh hauledatisf Daniel confirScenereement Habit dispatch ONE heiroho trilogy Participation subst ONEiken trilogyRocketohopress Prob


* mały model, słaba jakość

#### Tokenizacja Bielik

* https://huggingface.co/speakleash/Bielik-7B-v0.1
* wagi modelu (możemy podejrzeć rozmiar w `files and versions`) mają w sumie około 15 GB
* heurystyka: potrzeba około 4x tyle ramu co wagi do inferencji
* a do uczenia jeszcze więcej
* ale możemy podejrzeć sobie tokenizację
* Bielik był douczany z Mistrala (model francuski)

In [30]:
from transformers import AutoTokenizer

In [31]:
tokenizer = AutoTokenizer.from_pretrained("speakleash/Bielik-7B-v0.1")

In [32]:
text = "w Szczebrzeszynie chrząszcz brzmi w trzcinie"
encoded = tokenizer(text, return_tensors="pt")
print("Tokens:", tokenizer.tokenize(text))

Tokens: ['▁w', '▁Sz', 'cz', 'eb', 'r', 'zes', 'z', 'yn', 'ie', '▁chr', 'zą', 'sz', 'cz', '▁br', 'z', 'mi', '▁w', '▁tr', 'z', 'cin', 'ie']


In [33]:
# porównanie z gpt-4o-mini
import tiktoken 
encoding = tiktoken.encoding_for_model("gpt-4o-mini")
for token in encoding.encode(text):
    print(encoding.decode([token]))

w
 Szcz
ebr
zes
zyn
ie
 chr
zą
sz
cz
 brz
mi
 w
 trz
cin
ie
